In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
class FactVerificationDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        encoded_input = self.tokenizer.encode_plus(
            item['Claim'],
            item['evidence'],
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )
        label = item['label']
        return {
            'input_ids': encoded_input['input_ids'].squeeze(),
            'attention_mask': encoded_input['attention_mask'].squeeze(),
            'label': torch.tensor(label)
        }

In [ ]:
import json

# Step 1: Open the JSON file
with open('/content/results_5000 (1).json', 'r', encoding='utf-8') as file:
    # Step 2: Parse the JSON data
    training_data = json.load(file)

# Step 3: Access the data
print(training_data)

[{'Claim': 'Video makes infertility claim about development of GM crops\n', 'Most Relevant Sentence': 'A video featuring the founder of a far-right US-based website claims scientists can now genetically modify crops in order "to cause infertility in black people" by using a specific type of technology however This is misleading', 'label': 0}, {'Claim': "The Stormzy Prize-winner's debut We Are All Birds of Uganda is an ambitious multi-layered epic.", 'Most Relevant Sentence': 'It was praised for its deep storytelling and ability to celebrate love while also addressing pressing global issues', 'label': 2}, {'Claim': 'Dom Bess took five wickets on Day 1 of the 1st Test against Sri Lanka, but said he got lucky with some of the wickets and he could have bowled better.', 'Most Relevant Sentence': 'After their tons in the ICC World Test Championship Final against India at The Oval, Head has 884 points', 'label': 2}, {'Claim': 'Locals in Pakistan opposed tree plantation because it was un-Islam

In [ ]:
import json

# Step 1: Open the JSON file
with open('/content/test - Copy.json', 'r', encoding='utf-8') as file:
    # Step 2: Parse the JSON data
    testing_data = json.load(file)

# Step 3: Access the data
print(testing_data)

[{'Claim': 'Asif Zardari has announced support for Imran Khan, saying Shehbaz Sharif cannot lead Pakistan', 'evidence': 'Asif Zardari has not announced support for Khan, nor has he said Sharif cannot lead Pakistan', 'label': 0}, {'Claim': 'A video shows a Punjab Police officer “misbehaving” and “insulting” an older woman amid a recent crackdown by authorities following protests across Pakistan against the 9 May arrest of Imran Khan.', 'evidence': 'The video is not recent, nor does it have anything to do with the May 2023 crackdown by Punjab Police and they were not misbehaving and insulting an older women. ', 'label': 0}, {'Claim': "Video shows Akshay Kumar condemning Imran Khan's arrest", 'evidence': 'We found that the viral clip has been digitally altered by adding Kumar condemning the arrest of imran Khan as Akshay kumar was not condemn the arrest of imran khan', 'label': 0}, {'Claim': 'Former prime minister and Pakistan Tehreek-e-Insaf (PTI) chief Imran Khan has died.', 'evidence':

In [ ]:
dataset = FactVerificationDataset(training_data)

In [ ]:
# Step 4: Model architecture and fine-tuning
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Step 5: Train the model
optimizer = AdamW(model.parameters(), lr=2e-5)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)# Step 4: Model architecture and fine-tuning
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Step 5: Train the model
optimizer = AdamW(model.parameters(), lr=2e-5)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Average Loss: {average_loss}")


Epoch 1/2 - Average Loss: 0.38842618936738266
Epoch 2/2 - Average Loss: 0.127713384237676


In [ ]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
evaluation_dataset = FactVerificationDataset(testing_data)  # Replace eval_data with your evaluation dataset
evaluation_dataloader = DataLoader(evaluation_dataset, batch_size=16, shuffle=False)

In [ ]:
correct_predictions = 0
total_predictions = 0
predicted_labels_list = []  # List to store predicted labels
original_labels_list = []   # List to store original labels

for batch in evaluation_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_labels = torch.argmax(outputs.logits, dim=1)

    correct_predictions += (predicted_labels == labels).sum().item()
    total_predictions += len(labels)

    # Append predicted and original labels to their respective lists
    predicted_labels_list.extend(predicted_labels.cpu().numpy())
    original_labels_list.extend(labels.cpu().numpy())

accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy}")

# Now you can work with predicted_labels_list and original_labels_list outside the loop


Accuracy: 0.9722814498933902


In [ ]:
from sklearn.metrics import classification_report

# Assuming original_labels_list and predicted_labels_list are your data


# Get unique labels from the data
labels = sorted(set(original_labels_list + predicted_labels_list))

# Compute the classification report
class_report = classification_report(original_labels_list, predicted_labels_list, labels=labels)

# Print the classification report
print("Classification Report:")
print(class_report)


Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       172
           1       0.99      0.94      0.96       174
           2       0.98      0.99      0.98       123

    accuracy                           0.97       469
   macro avg       0.97      0.97      0.97       469
weighted avg       0.97      0.97      0.97       469

